In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arnaud58/landscape-pictures")

print("Path to dataset files:", path)

100%|██████████| 620M/620M [01:39<00:00, 6.51MB/s] 

Extracting files...


Path to dataset files: /Users/gabrielmindlberger/.cache/kagglehub/datasets/arnaud58/landscape-pictures/versions/2


In [5]:
import os

# Alle .jpg-Dateien im Ordner (ohne Sortierung)
files = [f for f in os.listdir(path) if f.lower().endswith(".jpg")]

total = len(files)
digits = len(str(total))  # Anzahl Ziffern

for idx, fname in enumerate(files, start=1):
    src = os.path.join(path, fname)
    new_name = f"{idx:0{digits}d}.jpg"  # z.B. 001.jpg, 002.jpg, …
    dst = os.path.join(path, new_name)
    os.rename(src, dst)

print(f"Renamed {total} files in '{path}'")

Renamed 4319 files in '/Users/gabrielmindlberger/.cache/kagglehub/datasets/arnaud58/landscape-pictures/versions/2'


In [ ]:
# rename files TO COLORIZED